In [1]:
import lightning
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks.model_checkpoint import ModelCheckpoint
import torch
import plotly.graph_objs as go
from sklearn.metrics import mean_absolute_percentage_error

In [2]:
from nepare.nn import NeuralPairwiseRegressor as NPR, FeedforwardNeuralNetwork as FNN
from nepare.data import PairwiseAugmentedDataset, PairwiseAnchoredDataset, PairwiseInferenceDataset
from nepare.inference import predict

In [3]:
TRAIN_MARKER = dict(mode="markers", marker=dict(color="red", opacity=0.2, size=4))
TRAIN_PRED = dict(mode="markers", marker=dict(color="red", opacity=0.4, symbol="diamond", size=4))
VAL_MARKER = dict(mode="markers", marker=dict(color="green", opacity=0.2, size=4))
VAL_PRED = dict(mode="markers", marker=dict(color="green", opacity=0.4, symbol="diamond", size=4))
TEST_MARKER = dict(mode="markers", marker=dict(color="blue", opacity=0.2, size=4))
TEST_PRED = dict(mode="markers", marker=dict(color="blue", opacity=0.4, symbol="diamond", size=4))

In [4]:
SEED = 1701
lightning.seed_everything(SEED)

Seed set to 1701


1701

In [5]:
X = torch.rand((200, 2), dtype=torch.float32)
y = torch.sin(8*X[:, 0]) + 3 * X[:, 1].pow(2)  # sin(x1) + 3 * x2^2
y = y.reshape(-1, 1)

In [6]:
train_idxs = torch.argwhere((X < 0.5).any(dim=1)).flatten()
val_idxs = train_idxs[0:int(len(train_idxs)*0.2)]
train_idxs = train_idxs[int(len(train_idxs)*0.2):]
test_idxs = torch.argwhere((X >= 0.5).all(dim=1)).flatten()

In [7]:
fig = go.Figure()
fig.update_layout(
    autosize=False,
    width=700,
    height=700,
)
fig.add_trace(go.Scatter3d(x=X[train_idxs, 0], y=X[train_idxs, 1], z=y[train_idxs].flatten(), **TRAIN_MARKER, name="Training"))
fig.add_trace(go.Scatter3d(x=X[val_idxs, 0], y=X[val_idxs, 1], z=y[val_idxs].flatten(), **VAL_MARKER, name="Validation"))
fig.add_trace(go.Scatter3d(x=X[test_idxs, 0], y=X[test_idxs, 1], z=y[test_idxs].flatten(), **TEST_MARKER, name="Testing"))
fig.show()

In [8]:
training_dataset = torch.utils.data.TensorDataset(X[train_idxs], y[train_idxs])
validation_dataset = torch.utils.data.TensorDataset(X[val_idxs], y[val_idxs])
testing_dataset = torch.utils.data.TensorDataset(X[test_idxs], y[test_idxs])
predict_dataset = torch.utils.data.TensorDataset(X[test_idxs])
train_loader = torch.utils.data.DataLoader(training_dataset, batch_size=8, shuffle=True)
val_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=8)
test_loader = torch.utils.data.DataLoader(testing_dataset, batch_size=8)
predict_loader = torch.utils.data.DataLoader(predict_dataset, batch_size=4)
fnn = FNN(2, 50, 3)
early_stopping = EarlyStopping(monitor="validation/loss")
model_checkpoint = ModelCheckpoint(monitor="validation/loss")
trainer = lightning.Trainer(max_epochs=50, log_every_n_steps=1, callbacks=[early_stopping, model_checkpoint])
trainer.fit(fnn, train_loader, val_loader)
fnn = FNN.load_from_checkpoint(model_checkpoint.best_model_path)  # reload best model based on early stopping
trainer.test(fnn, test_loader)
y_pred = torch.vstack(trainer.predict(fnn, predict_loader)).numpy()


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params | Mode 
--------------------------------------------
0 | fnn  | Sequential | 5.3 K  | train
--------------------------------------------
5.3 K     Trainable params
0         Non-trainable params
5.3 K     Total params
0.021     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/jackson/neural-pairwise-regression/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: PossibleUserWarning:

The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.

/home/jackson/neural-pairwise-regression/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: PossibleUserWarning:

The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.



Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

`Trainer.fit` stopped: `max_epochs=50` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/jackson/neural-pairwise-regression/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: PossibleUserWarning:

The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.



Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       testing/loss        │    1.2143949270248413     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/jackson/neural-pairwise-regression/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: PossibleUserWarning:

The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.



Predicting: |          | 0/? [00:00<?, ?it/s]

In [9]:
fig = go.Figure()
fig.update_layout(
    autosize=False,
    width=500,
    height=500,
)
# fig.add_trace(go.Scatter3d(x=X[train_idxs, 0], y=X[train_idxs, 1], z=y[train_idxs].flatten(), **TRAIN_MARKER, name="Training"))
# fig.add_trace(go.Scatter3d(x=X[val_idxs, 0], y=X[val_idxs, 1], z=y[val_idxs].flatten(), **VAL_MARKER, name="Validation"))
fig.add_trace(go.Scatter3d(x=X[test_idxs, 0], y=X[test_idxs, 1], z=y[test_idxs].flatten(), **TEST_MARKER, name="Testing"))
fig.add_trace(go.Scatter3d(x=X[test_idxs, 0], y=X[test_idxs, 1], z=y_pred.flatten(), **TEST_PRED, name="Predictions"))
fig.show()
print(mean_absolute_percentage_error(y[test_idxs], y_pred, sample_weight=y[test_idxs]))

0.4740895926952362


In [10]:
training_dataset = PairwiseAugmentedDataset(X[train_idxs], y[train_idxs], how='full')
validation_dataset = PairwiseAnchoredDataset(X[train_idxs], y[train_idxs], X[val_idxs], y[val_idxs], how='full')
# for metrics reported in the pairwise space
testing_dataset = PairwiseAnchoredDataset(X[train_idxs], y[train_idxs], X[test_idxs], y[test_idxs], how='full')
# for metrics reported in the absolute space
predict_dataset = PairwiseInferenceDataset(X[train_idxs], y[train_idxs], X[test_idxs], how='full')
train_loader = torch.utils.data.DataLoader(training_dataset, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=64)
test_loader = torch.utils.data.DataLoader(testing_dataset, batch_size=64)
predict_loader = torch.utils.data.DataLoader(predict_dataset, batch_size=4)

In [11]:
npr = NPR(2, 50, 3)
early_stopping = EarlyStopping(monitor="validation/loss")
model_checkpoint = ModelCheckpoint(monitor="validation/loss")

In [12]:
trainer = lightning.Trainer(max_epochs=50, log_every_n_steps=1, callbacks=[early_stopping, model_checkpoint])
trainer.fit(npr, train_loader, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params | Mode 
--------------------------------------------
0 | fnn  | Sequential | 5.4 K  | train
--------------------------------------------
5.4 K     Trainable params
0         Non-trainable params
5.4 K     Total params
0.022     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/jackson/neural-pairwise-regression/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: PossibleUserWarning:

The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.

/home/jackson/neural-pairwise-regression/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: PossibleUserWarning:

The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.



Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


In [13]:
npr = NPR.load_from_checkpoint(model_checkpoint.best_model_path)  # reload best model based on early stopping

In [14]:
trainer.test(npr, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/jackson/neural-pairwise-regression/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: PossibleUserWarning:

The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.



Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       testing/loss        │    0.16843527555465698    │
└───────────────────────────┴───────────────────────────┘

[{'testing/loss': 0.16843527555465698}]

In [15]:
y_pred, y_stdev = predict(npr, predict_loader, how="all")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/jackson/neural-pairwise-regression/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: PossibleUserWarning:

The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.



Predicting: |          | 0/? [00:00<?, ?it/s]

In [16]:
fig = go.Figure()
fig.update_layout(
    autosize=False,
    width=500,
    height=500,
)
fig.add_trace(go.Scatter3d(x=X[train_idxs, 0], y=X[train_idxs, 1], z=y[train_idxs].flatten(), **TRAIN_MARKER, name="Training"))
fig.add_trace(go.Scatter3d(x=X[val_idxs, 0], y=X[val_idxs, 1], z=y[val_idxs].flatten(), **VAL_MARKER, name="Validation"))
fig.add_trace(go.Scatter3d(x=X[test_idxs, 0], y=X[test_idxs, 1], z=y[test_idxs].flatten(), **TEST_MARKER, name="Testing"))
fig.add_trace(go.Scatter3d(x=X[test_idxs, 0], y=X[test_idxs, 1], z=y_pred, error_z=dict(
            type='data', # value of error bar given in data coordinates
            array=y_stdev,
            visible=True), **TEST_PRED, name="Prediction"))
fig.show()

In [17]:
print(mean_absolute_percentage_error(y[test_idxs], y_pred, sample_weight=y[test_idxs]))

0.14381085975775607
